In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2

def flann_match(des1, des2):
    des1 = np.float32(des1)
    des2 = np.float32(des2)
    
    # FLANN parameters
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    
    # Ratio test as per Lowe's paper
    good_matches = []
    for m, n in matches:
        if m.distance < 0.95 * n.distance:
            good_matches.append(m)

    return good_matches

def find_homography(kp1, kp2, matches, ransacReprojThreshold=5.0, maxIters=1000):
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, ransacReprojThreshold, maxIters=maxIters)
    
    return matches, mask



def draw_custom_matches(img1, kp1, img2, kp2, matches, mask):
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    nWidth = w1 + w2
    nHeight = max(h1, h2)
    hdif = (h2 - h1) // 2
    new_img = np.zeros((nHeight, nWidth, 3), dtype=np.uint8)
    new_img[hdif:hdif + h1, :w1] = img1
    new_img[:h2, w1:w1 + w2] = img2
    
    # 点旁边加序号
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_color = (255, 0, 0) 
    font_thickness = 2

    # 解析出匹配点
    kp1_final = []
    kp2_final = []

    for i, m in enumerate(matches):
        if mask[i]:  
            # 
            pt1 = kp1[m.queryIdx].pt
            pt2 = kp2[m.trainIdx].pt

            # 
            kp1_final.append(pt1)
            kp2_final.append(pt2)

            # 可视化匹配点和连线
            pt1_draw = (int(pt1[0]), int(pt1[1]) + hdif)
            pt2_draw = (int(pt2[0]) + w1, int(pt2[1]))
            cv2.circle(new_img, pt1_draw, 5, (255, 0, 0), -1)
            cv2.circle(new_img, pt2_draw, 5, (0, 255, 0), -1)
            cv2.line(new_img, pt1_draw, pt2_draw, (255, 255, 0), 2)

    plt.imshow(new_img)
    plt.axis('off')
    plt.show()

    kp1_final = np.array(kp1_final)
    kp2_final = np.array(kp2_final)
    
    return kp1_final, kp2_final

def calculate_distance_proportions(kp1_final, kp2_final, T1=10):
    for i in range(len(kp1_final)):
        distances1 = np.linalg.norm(kp1_final - kp1_final[i], axis=1)
        distances2 = np.linalg.norm(kp2_final - kp2_final[i], axis=1)
        
        absolute_differences = np.abs(distances1 - distances2)
        
        proportion_greater_than_T1 = np.mean(absolute_differences > T1)

        print(f" {i}: {proportion_greater_than_T1:.2f}")

# 利用判断的点修改mask
def calculate_new_mask(kp1_final, kp2_final,mask, T1=10,T2=0.5):
    mask_new= mask
    indices = np.where(mask_new == 1)[0]
    for i in range(len(kp1_final)):
        distances1 = np.linalg.norm(kp1_final - kp1_final[i], axis=1)
        distances2 = np.linalg.norm(kp2_final - kp2_final[i], axis=1)
        absolute_differences = np.abs(distances1 - distances2)
        proportion_greater_than_T1 = np.mean(absolute_differences > T1)
        if proportion_greater_than_T1>T2:
            mask_new[indices[i]]=0
    return mask_new

In [1]:
# ----------------------------------开始计算-----------------------------------------    
basepath = r'F:\PC2-Data\UAV_GE1\Pairs_new4\JJ\U2'
result_path = r'F:\PC2-Data\UAV_GE1\Pairs_new4\results\JJ\JJ_LNIFT_1.txt'
with open(result_path, 'w') as result_file:
    for folder in os.listdir(basepath):
        folder_path = os.path.join(basepath, folder)
        print('--------------------------------------------------------')
        print(folder)
        if os.path.isdir(folder_path):
            # Load data and images
            LNIFT_locs1_path = os.path.join(folder_path, 'LNIFT_kpts1.txt')
            LNIFT_locs2_path = os.path.join(folder_path, 'LNIFT_kpts2.txt')
            LNIFT_des1_path = os.path.join(folder_path, 'LNIFT_des1.txt')
            LNIFT_des2_path = os.path.join(folder_path, 'LNIFT_des2.txt')
            image1_path = os.path.join(folder_path, '1360.png')
            image2_path = os.path.join(folder_path, 'Ref.jpg')
            # Data loading
            image1 = cv2.imread(image1_path)
            image2 = cv2.imread(image2_path)
            LNIFT_locs1_data = np.loadtxt(LNIFT_locs1_path)
            LNIFT_locs2_data = np.loadtxt(LNIFT_locs2_path)
            LNIFT_des1_data = np.loadtxt(LNIFT_des1_path)
            LNIFT_des2_data = np.loadtxt(LNIFT_des2_path)
            # 数据转换
            locs1 = LNIFT_locs1_data[:,0:2]
            locs2 = LNIFT_locs2_data[:,0:2]
            des1 = LNIFT_des1_data
            des2 = LNIFT_des2_data
            kp1 = [cv2.KeyPoint(x, y, 1) for x, y in locs1]
            kp2 = [cv2.KeyPoint(x, y, 1) for x, y in locs2]

            # 开始匹配
            good_matches = flann_match(des1, des2)
            for i in range(4):
                matches, mask = find_homography(kp1, kp2, good_matches, ransacReprojThreshold=3+2*i, maxIters=5000)
                kp1_final,kp2_final = draw_custom_matches(image1, kp1, image2, kp2, good_matches, mask.ravel().tolist())
                mask_new = calculate_new_mask(kp1_final, kp2_final,mask,T1=10,T2=0.5)
                kp1_final,kp2_final = draw_custom_matches(image1, kp1, image2, kp2, good_matches, mask_new.ravel().tolist())
                if np.sum(mask_new)>=5:
                    break
                if i==3:
                    mask_new=0
                print('-------------------------------------------------')
            result_file.write(f"{np.sum(mask)} {np.sum(mask_new)}\n")

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\PC2-Data\\UAV_GE1\\Pairs_new4\\results\\JJ\\JJ_LNIFT_1.txt'